In [1]:
from splinter import Browser
from selenium.common.exceptions import InvalidElementStateException
from selenium.common.exceptions import ElementNotInteractableException
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd
import html5lib
import lxml
import shutil
import time
import urllib3
import requests
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
import json
import os
from os import path

In [2]:
codes = []
codes.append('27606')   # just hard code one in
code = codes[0]   # set the variable for zip_and_click
global browser
forecast_type = str   # so I can add the dict field in the function argument
data = {}
host = 'localhost'
port = 27017
# inititial website to visit
url = 'https://weather.com'

In [15]:
def chrome():
    '''
    Finds the chromedriver in the system and creates a Chrome browser
    '''
    print('in chrome')
    executable_path = {'executable_path': shutil.which('chromedriver')}
    browser = Browser('chrome', **executable_path)
    return(browser)


def zip_and_click(code):
    '''
    Enter zip codes into the search bar on weather.com and click the first result.
    No return, just leaves browser at the first data page.
    '''
    print('in zip_and_click()')
    import time
    
    filled = False
    clicked = False
    n=0
    
    inputs = browser.find_by_tag('input') # get the search box reference
    print('inputs set')
    search_box = inputs[0]
    while not filled:
        print('trying to fill search_box')
        try:
            search_box.fill(code)
            filled = True
            print('filled')
        except InvalidElementStateException:
            time.sleep(1)
    while not clicked:
        print('while condition satisfied')
        print(clicked)
        try:
            print('checking for the zipcode in search_box')
            print(search_box.value)
            if browser.find_by_text(f'{code}'):
                print("found the zip code text")
            else:
                print('did not get the zip code text')
            print('trying to click search result')
            browser.click_link_by_partial_href('/weather/today/l')
            print('no exception this time')
            clicked = True
            print(clicked)
        except ElementNotInteractableException:
            print('exception found....waiting...')
            n+=1
            print(f'n={n}')
            time.sleep(.5)
            if n==5:
                print('it is not clicking')
                return
    return(search_box)


def wind_dict(wind_str):
    ''' handle the cases where the wind value is simply "calm" rather than direction and speed '''
    print('in wind_dict()')
    if wind_str == 'Calm':
        return {
            'speed': 0,
            'direction': None
        }
    else:
        return {
            'speed': wind_str.split()[1].lower(),
            'direction': wind_str.split()[0].lower()
        }
    
    
def to_json(data, code):
    ''' Store the collected data as a json file in the case that the database
        is not connected or otherwise unavailable. 
    '''
    print('in to_json()')
    collection = json.dumps(data)
    directory = os.getcwd()
    path = path.join(directory, 'Data', f'{code}.json')
    Data = open(path, 'a')
    Data.write(collection)
    Data.close()
    return


def scrape_now(browser):
    ''' Get observed weather data from the current weather page '''
    print('in scrape_now()')
    right_now = {}
    divs = browser.find_by_tag('div').first
    nowcard = divs.find_by_tag('section.today_nowcard-container')[0].value
    now = nowcard.split('\n')
    
    right_now.update({'as_of': [x.lower() for  x in now[1].split()[3:5]],
                 'observations': {'temp F': {'measured': now[2].lower(),
                                            'feels_like': [x.lower() for x in now[4][11:]]
                                            },
                                 'description': now[3].lower(),
                                 'uv_index': now[6].split()[3].lower(),
                                 'wind': wind_dict(now[10]),
#                                     {'speed mph': now[10].split()[1].lower(),
#                                          'direction': now[10].split()[0].lower()
#                                          },
                                 'humidity': now[12].lower(),
                                 'dew_point': now[14].lower(),
                                 'pressure': {'in': now[16].split()[0].lower(),
                                             'change': ''
                                             },
                                  'visibility mi': now[18].lower()
                                 }
                 })
    # Store the data
    data['right_now'] = right_now
#     to_json(data=right_now, code)
    return


def scrape_next36(browser):
    ''' Get the "at a glance" data from the next 36 hours '''
    print('scrape next36()')

    next_36 = {}
    
    # Pulling data from the "next 36 hours" overview displays. There are 5 sections
    # that are pulled from. Each section gets a different label.
    hour_code = {'0':'now',
                '1':'at_start',
                '2':'at_12',
                '3':'at_24',
                '4':'at_36'
                }
    print('start next36 loop')
    for i in range(5):
        try:
            daypart = browser.find_by_tag(f'div#daypart-{i}')
            daypart.click()
        except ElementNotInteractableException:
            print('ElementNotInteractableException occured: breaking loop.')
            break
        snapshot = browser.find_by_tag(f'div#daypart-{i}')
        snap_values = snapshot.value
#         print(snap_values)
        details = browser.find_by_tag('span.wx-detail-value')
        detaillist = [details[j].value for j in range(4)]

        #store the data
        next_36.update({hour_code[str(i)]: {'hour': 'CALCULATED HOUR OF PREDICTION',
                                            'condition': snap_values.split('\n')[1].lower(),
                                            'high/low': snap_values.split('\n')[2].lower(),
                                            'temp_f': snap_values.split('\n')[3].lower(),
                                            'chance_precip': snap_values.split('\n')[4].lower(),
                                            'description': browser.find_by_id(f'dp{i}-details-narrative').first.value.lower(),
                                            'wind': wind_dict(detaillist[0]),
                                            'humidity': detaillist[1].lower(),
                                            'uv_index': detaillist[2].lower(),
                                            'sun': {'rise': detaillist[3].lower(),
                                                    'set': detaillist[3].lower()
                                                   }
                                              }
                      })
        time.sleep(.5)
    data['next_36'] = next_36
#     to_json(data=next_36, code)
    return


def goto_hourly(forecast_type, browser):
    ''' Take the browser to the hourly numbers '''
    print('in goto_hourly()')
    browser.find_by_text('Hourly').click()
    time.sleep(.5)
    n=0
    while browser.is_text_present('Next 8 Hours'):
        n+=1
        browser.find_by_text('Next 8 Hours').click()
        time.sleep(.5)
    else:
        print(f'got {8*n} hours of forcast')
    return


def get_hourly(forecast_type, browser):
    goto_hourly(forecast_type, browser)
    df = get_tables(forecast_type, browser)
    df = shift_col(df)
    return(df)


def get_tenday(forecast_type, browser):
    goto_tenday(forecast_type, browser)
    df = get_tables(forecast_type, browser)
    shift_col(df)
    return(df)

def get_tables(forecast_type, browser):
    ''' Get the tables from the current page and turn to pandas df's '''
    print('in get_tables')
    url = browser.url
    r = requests.get(url)
    dfs = pd.read_html(r.text)
    df = dfs[0]
    d_table = df.to_json(orient='index')
    data[forecast_type] = d_table
    return(df)


def shift_col(df):
    ''' Realign dataframe columns by storing the column names, deleting 
        the garbage column, renameing columns to their original names.
    '''
    cols = df.columns[:-1]
    df = df.drop('Day', 1)
    df.columns = cols
    return(df)


def goto_tenday(browser):
    ''' Get the ten day forcast '''
    print('in goto_tenday()')
    tenday = browser.find_by_text('10 Day Forecast')
    if tenday:
        tenday[1].click()
    time.sleep(1)
    return


def check_db_access(str: host, port):
    '''A check that there is write access to the database'''
    print('in db_check()')
   
    client = MongoClient(host=host, port=port)
    try:
        # The ismaster command is cheap and does not require auth.
        client.admin.command('ismaster')
        print('Connection made')
    except ConnectionFailure:
        print("Server not available")
        return

    # check the database connections
        # Get a count of the number of databases at the connection (accessible through that port)
        # before attempting to add to it
    db_count_pre = len(client.list_database_names())
        # Add a database and collection
    db = client.test_db
    col = db.test_col

    # Insert something to the db
    post = {'name':'Chuck VanHoff',
           'age':'38',
           'hobby':'gardening'
           }
    col.insert_one(post)

        # Get a count of the databases after adding one
    db_count_post = len(client.list_database_names())

    if db_count_pre-db_count_post>=0:
        print('Your conneciton is flipped up')
    else:
        print('You have write access')

    client.drop_database(db)
#     client.close()
    return(client)


def load(data):
    ''' Load the data to the database if possible, otherwise write to json file '''
    print('in load()')
    if type(data) == dict:
        try:
            db = client.forcast
            col = db.code
            col.insert_one(data)
            print('Inserted to database')
        except InvalidDocument:
            to_json(data, code)
            client.close()
            print('Wrote to json')
    else:
#         load = df.to_json(orient='index')
        to_json(load, code)
    return

In [4]:
shutil.which('chromedriver')

'/Users/mastacow/data/forcast-forcast/env/bin/chromedriver'

In [7]:
for i in range(5):
        try:
            daypart = browser.find_by_tag(f'div#daypart-{i}')
            daypart.click()
        except ElementNotInteractableException:
            print('ElementNotInteractableException occured: breaking loop.')
            break
        snapshot = browser.find_by_tag(f'div#daypart-{i}')
        snap_values = snapshot.value
#         print(snap_values)
        details = browser.find_by_tag('span.wx-detail-value')
        detaillist = [details[j].value for j in range(4)]
snap_values

'WED NIGHT\nPARTLY CLOUDY\nLOW\n28°\n0%'

In [9]:
snap_values.split('\n')

['WED NIGHT', 'PARTLY CLOUDY', 'LOW', '28°', '0%']

In [10]:
# 'chance_precip': snap_values.split('\n')[4].lower(),
snap_values.split('\n')[4].lower()

'0%'

In [44]:
browser.quit()

In [16]:
# client = check_db_access(host, port)
# browser = chrome()
# browser.visit(url)
# zip_and_click(code)
# scrape_now(browser)
# scrape_next36(browser)
get_hourly('hourly', browser)
get_tenday('ten_day', browser)

load(data)

in goto_hourly()
got 40 hours of forcast
in get_tables


KeyError: "['Day'] not found in axis"

In [118]:
browser.quit()

In [85]:
browser = chrome()

In [11]:
browser.visit(url)

In [12]:
zip_and_click(code)

in zip_and_click()
inputs set
trying to fill search_box
filled
while condition satisfied
False
checking for the zipcode in search_box
27606
did not get the zip code text
trying to click search result
no exception this time
True


In [88]:
browser.click_link_by_partial_href('/weather/today/l')


In [89]:
right_now = scrape_now(browser)
# pprint(right_now)

In [90]:
next_36 = scrape_next36(browser)
# pprint(next_36)

start next36 loop
TODAY
CLOUDY
HIGH
60°
10%
TONIGHT
CLOUDY
LOW
58°
10%
TUE
PM SHOWERS
HIGH
69°
50%
TUE NIGHT
RAIN
LOW
41°
80%
WED
AM LIGHT RAIN
HIGH
45°
70%


In [91]:
goto_hourly(browser)

got 32 hours of forcast


In [17]:
# A line that will click out of an advertisement
browser.find_by_tag('a#bx-close-inside-970188')[0].click()

In [36]:
hourly = get_tables('hourly', browser)
# hourly

in get_tables


In [37]:
goto_tenday(browser)

in goto_tenday()


In [38]:
df = get_tables('ten_day', browser)
df

in get_tables


,Day,Description,High / Low,Precip,Wind,Humidity,Unnamed: 6
0,NaN,TonightDEC 9,Cloudy,--58°,10%,SSW 9 mph,89%
1,NaN,TueDEC 10,PM Showers,71°41°,70%,SSW 12 mph,80%
2,NaN,WedDEC 11,AM Light Rain,46°28°,70%,N 8 mph,71%
3,NaN,Thu DEC 12,Mostly Sunny,42°31°,0%,NE 10 mph,50%
4,NaN,FriDEC 13,Showers,50°45°,40%,NE 8 mph,82%
5,NaN,SatDEC 14,Cloudy,57°43°,20%,WSW 6 mph,88%
6,NaN,SunDEC 15,Sunny,56°37°,10%,W 9 mph,70%
7,NaN,Mon DEC 16,Mostly Sunny,58°44°,20%,S 5 mph,68%
8,NaN,TueDEC 17,Showers,54°37°,40%,NNW 6 mph,76%
9,NaN,WedDEC 18,Showers,47°30°,40%,N 10 mph,64%


Index(['Day', 'Description', 'High / Low', 'Precip', 'Wind', 'Humidity'], dtype='object')
    Description     High / Low  Precip Wind    Humidity Unnamed: 6
0  TonightDEC 9         Cloudy   --58°  10%   SSW 9 mph        89%
1     TueDEC 10     PM Showers  71°41°  70%  SSW 12 mph        80%
2     WedDEC 11  AM Light Rain  46°28°  70%     N 8 mph        71%
3    Thu DEC 12   Mostly Sunny  42°31°   0%   NE 10 mph        50%
4     FriDEC 13        Showers  50°45°  40%    NE 8 mph        82%


,Day,Description,High / Low,Precip,Wind,Humidity
0,TonightDEC 9,Cloudy,--58°,10%,SSW 9 mph,89%
1,TueDEC 10,PM Showers,71°41°,70%,SSW 12 mph,80%
2,WedDEC 11,AM Light Rain,46°28°,70%,N 8 mph,71%
3,Thu DEC 12,Mostly Sunny,42°31°,0%,NE 10 mph,50%
4,FriDEC 13,Showers,50°45°,40%,NE 8 mph,82%


In [32]:
data

{'right_now': {'as_of': ['pm', 'est'],
  'observations': {'temp F': {'measured': '48°',
    'feels_like': ['4', '7', '°']},
   'description': 'mostly cloudy',
   'uv_index': 'of',
   'wind': {'speed': '5', 'direction': 'ne'},
   'humidity': '55%',
   'dew_point': '33°',
   'pressure': {'in': '30.41', 'change': ''},
   'visibility mi': '10.0 mi'}},
 'next_36': {'now': {'hour': 'CALCULATED HOUR OF PREDICTION',
   'condition': 'mostly cloudy',
   'high/low': 'high',
   'temp_f': '53°',
   'chance_precip': '20%',
   'description': 'a mix of clouds and sun early followed by cloudy skies this afternoon. high 53f. winds ene at 5 to 10 mph.',
   'wind': {'speed': '7', 'direction': 'ene'},
   'humidity': '58%',
   'uv_index': '3 of 10',
   'sun': {'rise': '7:11 am5:01 pm', 'set': '7:11 am5:01 pm'}},
  'at_start': {'hour': 'CALCULATED HOUR OF PREDICTION',
   'condition': 'showers',
   'high/low': 'low',
   'temp_f': '44°',
   'chance_precip': '60%',
   'description': 'cloudy with showers. low 44

In [40]:
load(data)

in load()
Inserted to database


In [48]:
# db = client.forcast
# col = db.code
post_id = col.ObjectId('5de871afff395937db2f6f30')
print(post_id)
# col.count_documents({'_id': post_id})

TypeError: 'Collection' object is not callable. If you meant to call the 'ObjectId' method on a 'Collection' object it is failing because no such method exists.

In [41]:
for x in col.find({}):
    pprint(x)

{'_id': ObjectId('5de6575e08b5184ef0c66939')}
{'_id': ObjectId('5de79796cad08f0bd0c801d8')}
{'_id': ObjectId('5de871afff395937db2f6f30'),
 'hourly': '{"columns":["Time","Description","Temp","Feels","Precip","Humidity","Wind","Unnamed: '
           '7"],"index":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],"data":[[null,"9:45 '
           'pmWed","Clear","44\\u00b0","44\\u00b0","0%","60%","W 2 '
           'mph"],[null,"10:00 '
           'pmWed","Clear","43\\u00b0","43\\u00b0","0%","60%","W 2 '
           'mph"],[null,"11:00 '
           'pmWed","Clear","42\\u00b0","42\\u00b0","0%","60%","WSW 1 '
           'mph"],[null,"12:00 '
           'amThu","Clear","42\\u00b0","42\\u00b0","0%","62%","WSW 2 '
           'mph"],[null,"1:00 '
           'amThu","Clear","41\\u00b0","41\\u00b0","0%","63%","WSW 2 '
           'mph"],[null,"2:00 '
           'amThu","Clear","42\\u00b0","39\\u00b0","0%","62%","W 4 '
           'mph"],[null,"3:00 '
           'amThu","Clear","41\\u00b0","38\\u00b0","0%","62%",

                     'temp_f': '29°',
                     'uv_index': '0 of 10',
                     'wind': {'direction': 'ene', 'speed': '2'}}},
 'right_now': {'as_of': ['pm', 'est'],
               'observations': {'description': 'clear',
                                'dew_point': '28°',
                                'humidity': '71%',
                                'pressure': {'change': '', 'in': '30.44'},
                                'temp F': {'feels_like': ['3', '4', '°'],
                                           'measured': '36°'},
                                'uv_index': 'of',
                                'visibility mi': '9.0 mi',
                                'wind': {'direction': 'w', 'speed': '3'}}},
 'ten_day': '{"0":{"Day":null,"Description":"TonightDEC 7","High \\/ '
            'Low":"Clear","Precip":"--29\\u00b0","Wind":"0%","Humidity":"ENE 2 '
            'mph","Unnamed: 6":"79%"},"1":{"Day":null,"Description":"Sun DEC '
            '8","High \\/

In [35]:
time.time()

1575778063.3095932

In [20]:
goto_hourly(browser)

1
got next 8 hour spotted
clicked and waiting
end n=1 th pass
2
got next 8 hour spotted
clicked and waiting
end n=2 th pass
3
got next 8 hour spotted
clicked and waiting
end n=3 th pass
4
got next 8 hour spotted
clicked and waiting
end n=4 th pass
5
got next 8 hour spotted
clicked and waiting
end n=5 th pass


In [28]:
browser.quit()

In [55]:
now

['ADVANCE, NC',
 'as of 11:27 pm EST',
 '43°',
 'CLEAR',
 'feels like 41°',
 'H -- L 35°',
 'UV Index 0 of 10',
 'Holiday Week Travel Forecast',
 'RIGHT NOW',
 'Wind',
 'NNW 4 mph',
 'Humidity',
 '57%',
 'Dew Point',
 '29°',
 'Pressure',
 '30.31 in',
 'Visibility',
 '10.0 mi']

In [ ]:
if now[10]

In [57]:
from pprint import pprint
pprint(next_36['now'])
# next_36

{'chance_precip': '0%',
 'condition': 'MOSTLY CLEAR',
 'description': 'Clear to partly cloudy. Low near 35F. Winds light and '
                'variable.',
 'high/low': 'LOW',
 'hour': 'CALCULATED HOUR OF PREDICTION',
 'humidity': '68%',
 'sun': {'rise': '7:10 am5:08 pm', 'set': '7:10 am5:08 pm'},
 'temp_f': '35°',
 'uv_index': '0 of 10',
 'wind': {'direction': 'NNE', 'speed': '4'}}


In [22]:
get_table(browser)

[    Time  Description        Temp Feels Precip Humidity Wind Unnamed: 7
 0    NaN  12:30 pmSat      Cloudy   48°    48°       0%  86%  SSE 2 mph
 1    NaN   1:00 pmSat      Cloudy   49°    49°       0%  85%  SSE 2 mph
 2    NaN   2:00 pmSat      Cloudy   50°    50°       0%  81%  SSE 2 mph
 3    NaN   3:00 pmSat      Cloudy   50°    50°      10%  78%  SSE 3 mph
 4    NaN   4:00 pmSat      Cloudy   50°    50°      25%  77%   SE 2 mph
 5    NaN   5:00 pmSat        Rain   50°    50°      70%  80%  ESE 1 mph
 6    NaN   6:00 pmSat     Showers   49°    49°      65%  89%  ESE 1 mph
 7    NaN   7:00 pmSat        Rain   48°    48°      60%  93%   SE 2 mph
 8    NaN   8:00 pmSat  Light Rain   47°    47°      65%  93%   SE 2 mph
 9    NaN   9:00 pmSat  Light Rain   47°    47°      65%  94%  ESE 2 mph
 10   NaN  10:00 pmSat        Rain   47°    47°      85%  96%  ESE 2 mph
 11   NaN  11:00 pmSat        Rain   47°    47°      90%  95%   SE 2 mph
 12   NaN  12:00 amSun        Rain   47°    46°    

In [36]:
browser.back()

In [35]:
len(dfs)

1

In [ ]:
# snapshot = browser.find_by_tag('div#daypart-0').value
# snaplist = snapshot.split('\n')
# details = browser.find_by_tag('span.wx-detail-value')
# detailslist = [details[i].value for i in range(4)]
# print(snaplist)
# detailslist

In [49]:
print(snaplist)

['TONIGHT', 'RAIN', 'LOW', '42°', '90%']


None


NNW 10 mph
86%
0 of 10
6:43 am5:13 pm


'TODAY\nHIGH\n70°\n10%'

'A clear sky. Scattered frost possible. Low around 30F. Winds light and variable.'

In [18]:
browser.quit()

1

'Today'

got input box


1

In [25]:
hour_code = {'0':'now',
            '1':'at_start',
            '2':'at_12',
            '3':'at_24',
            '4':'at_36'
}
print(hour_code['0'])
for i in range(5):
    print(hour_code[str(i)])
    print(f'looking for {hour_code[str(i)]} snapshot')

now
now
looking for now snapshot
at_start
looking for at_start snapshot
at_12
looking for at_12 snapshot
at_24
looking for at_24 snapshot
at_36
looking for at_36 snapshot


In [ ]:
### SEARCH FOR THE TABLE THAT HOLDS THE SEARCH RESULTS ###